<a href="https://colab.research.google.com/github/srishti-git1110/Simple_NN_in_PyTorch/blob/main/FFNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is just to get a hands on demo of pytorch for ones who know what neural networks are & their working in theory, and want to implement their theoretical knowledge in pytorch.
You can run this nb on a gpu as well as cpu.

We construct a very short network here working on MNIST data.

In [1]:
import torch.nn as nn #ffnn, cnn, rnn etc
import torch.optim as optim #optimisers like sgd etc
import torch.nn.functional as f #activ funcs
from torch.utils.data import DataLoader #minibatch train test etc
import torchvision.datasets as datasets #std datasets
import torchvision.transforms as transforms #transformations to perform on dataset


In [6]:
import torch
x = torch.rand(size=(64,10))
_, predictions = x.max(1)
print(x.max(1))


torch.return_types.max(
values=tensor([0.8831, 0.9590, 0.7447, 0.8181, 0.9913, 0.9424, 0.8740, 0.6849, 0.9429,
        0.9806, 0.8797, 0.8810, 0.9785, 0.9850, 0.8550, 0.9733, 0.9678, 0.7914,
        0.7959, 0.8727, 0.9407, 0.9979, 0.6810, 0.8131, 0.9454, 0.9641, 0.7145,
        0.8946, 0.9875, 0.9655, 0.8828, 0.9104, 0.9815, 0.8680, 0.9967, 0.9163,
        0.8877, 0.9889, 0.9253, 0.8084, 0.7278, 0.9804, 0.9271, 0.8693, 0.9391,
        0.8525, 0.6139, 0.9333, 0.9073, 0.8948, 0.8275, 0.8844, 0.9992, 0.9056,
        0.8856, 0.7761, 0.8785, 0.8311, 0.9673, 0.7846, 0.9639, 0.9899, 0.9435,
        0.8852]),
indices=tensor([2, 9, 4, 9, 3, 8, 4, 2, 2, 4, 0, 8, 9, 6, 2, 6, 3, 4, 7, 1, 4, 6, 9, 2,
        6, 1, 0, 9, 4, 4, 9, 9, 3, 2, 1, 5, 3, 3, 6, 9, 3, 0, 7, 4, 9, 4, 0, 7,
        8, 6, 4, 8, 7, 3, 8, 1, 9, 1, 2, 4, 2, 4, 9, 3]))


In [9]:


class NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(NN, self).__init__()
    self.fc1 = nn.Linear(in_features=input_size, out_features=50)
    self.fc2 = nn.Linear(in_features=50, out_features=num_classes)

  def forward(self, x): #x is the data examples*features
    x = f.relu(self.fc1(x)) 
    x = self.fc2(x)
    return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size = 784
num_classes = 10
learning_rate = .001
batch_size = 64
epochs = 1

#load data
train_set = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)

test_set = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

#initialize
model = NN(input_size, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

#train
for epoch in range(epochs):
  for batch_num, (data, targets) in enumerate(train_loader):
    data = data.to(device=device)
    targets = targets.to(device=device)
    data = data.reshape(data.shape[0], -1)
    #forward
    scores = model(data)
    loss = criterion(scores, targets)

    #backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

def check_accuracy(loader, model):
  if loader.dataset.train:
    print("checking accuracy on train data")
  else:
    print("checking accuracy on test data")
  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device)
      y = y.to(device=device)
      x = x.reshape(x.shape[0], -1)

      scores = model(x) #64*10
      _, predictions = scores.max(1) #probs, indices
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)
    print(f'Accuracy {float(num_correct)/float(num_samples)*100:.2f}')
  model.train()
  

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

checking accuracy on train data
Accuracy 93.38
checking accuracy on test data
Accuracy 93.36
